In [1]:
#import required packages
#basics
import pandas as pd 
import numpy as np

#misc
import gc
import time
import warnings

#stats
#from scipy.misc import imread
from scipy import sparse
import scipy.stats as ss

In [2]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
#import matplotlib_venn as venn

In [3]:
#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords


In [4]:
#import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer

In [5]:
#FeatureEngineering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import  textblob
#import xgboost
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

from textblob import TextBlob
from nltk.stem import PorterStemmer
import nltk
#nltk.download('wordnet')
from textblob import Word

In [6]:
#settings
start_time=time.time()
color = sns.color_palette()
sns.set_style("dark")
eng_stopwords = set(stopwords.words("english"))
warnings.filterwarnings("ignore")

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

%matplotlib inline

import nltk
nltk.download('stopwords')

In [134]:
%config Completer.use_jedi=False

In [135]:
data = pd.read_excel(r"E:\DSP\DSP_22nd Nov\Machine Learning\FInal Class\Case Studies & FInal Projects\5\BankReviews.xlsx")
data.head()

,Date,Stars,Reviews,BankName
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage


In [136]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
Date        505 non-null datetime64[ns]
Stars       505 non-null int64
Reviews     505 non-null object
BankName    505 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 15.9+ KB


In [137]:
data.Stars.value_counts()

5    410
1     95
Name: Stars, dtype: int64

In [138]:
data['sentiment'] = data["Reviews"].apply(lambda x: TextBlob(x).sentiment.polarity )

In [142]:
data["sentiment_type"] = np.where((data.sentiment < -0.1),-1,np.where((data.sentiment > 0.1),1,0))

In [144]:
data.head(20)

,Date,Stars,Reviews,BankName,sentiment,sentiment_type
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,0.533333,1
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,0.453333,1
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,-0.033231,0
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,0.093740,0
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,0.125000,1
5,2016-12-20,5,I had the pleasure of working with Wyndham Cap...,Wyndham Capital Mortgage,0.345833,1
6,2017-12-17,5,My experience with Mattison was beyond greatly...,Wyndham Capital Mortgage,0.313281,1
7,2016-08-16,5,Patrick answered all my questions by email imm...,Wyndham Capital Mortgage,0.238435,1
8,2017-09-04,5,I loved working with this group of people! The...,Wyndham Capital Mortgage,0.362500,1
9,2016-03-22,5,Great web interface for both the loan applicat...,Wyndham Capital Mortgage,0.566667,1


### Exploratory Analysis

#### Creating Additional columns to explore the data well.

In [145]:
import re

data['Reviews'] = data['Reviews'].astype(str)
data['count_sent']=data["Reviews"].apply(lambda x: len(re.findall("\n",str(x)))+1)

#Word count in each comment:
data['count_word']=data["Reviews"].apply(lambda x: len(str(x).split()))

#Unique word count
data['count_unique_word']=data["Reviews"].apply(lambda x: len(set(str(x).split())))

#Letter count
data['count_letters']=data["Reviews"].apply(lambda x: len(str(x)))

#Word density

data['word_density'] = data['count_letters'] / (data['count_word']+1)

#punctuation count
data["count_punctuations"] =data["Reviews"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

#upper case words count
data["count_words_upper"] = data["Reviews"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

#upper case words count
data["count_words_lower"] = data["Reviews"].apply(lambda x: len([w for w in str(x).split() if w.islower()]))

#title case words count
data["count_words_title"] = data["Reviews"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

#Number of stopwords
data["count_stopwords"] = data["Reviews"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

#Average length of the words
data["mean_word_len"] = data["Reviews"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

#Number of numeric
data['numeric'] = data['Reviews'].apply(lambda x :len([x for x in x.split() if x.isdigit()]))

#Number of alphanumeric
data['alphanumeric'] = data['Reviews'].apply(lambda x :len([x for x in x.split() if x.isalnum()]))

#Number of alphabetics
data['alphabetetics'] = data['Reviews'].apply(lambda x :len([x for x in x.split() if x.isalpha()]))

#Number of alphabetics
data['Spaces'] = data['Reviews'].apply(lambda x :len([x for x in x.split() if x.isspace()]))

#Number of Words ends with
data['words_ends_with_et'] = data['Reviews'].apply(lambda x :len([x for x in x.lower().split() if x.endswith('et')]))

#Number of Words ends with
data['words_start_with_no'] = data['Reviews'].apply(lambda x :len([x for x in x.lower().split() if x.startswith('no')]))

# Count the occurences of all words
data['wordcounts'] = data['Reviews'].apply(lambda x :dict([ [t, x.split().count(t)] for t in set(x.split()) ]))


In [146]:
data.head()

,Date,Stars,Reviews,BankName,sentiment,sentiment_type,count_sent,count_word,count_unique_word,count_letters,...,count_words_title,count_stopwords,mean_word_len,numeric,alphanumeric,alphabetetics,Spaces,words_ends_with_et,words_start_with_no,wordcounts
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,0.533333,1,1,19,19,126,...,5,5,5.684211,0,15,15,0,0,0,"{'Thank': 1, 'and': 1, 'you!': 1, 'helped': 1,..."
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,0.453333,1,1,25,23,159,...,5,10,5.400000,0,21,21,0,0,0,"{'Thank': 1, 'much': 1, 'service,': 1, 'produc..."
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,-0.033231,0,1,79,64,462,...,14,31,4.860759,0,73,73,0,0,1,"{'money.': 1, 'Brad': 1, 'past': 1, '&': 1, 'a..."
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,0.093740,0,1,108,78,605,...,18,49,4.611111,0,101,101,0,0,1,"{'money.': 1, 'was': 1, 'dealing': 2, 'extra':..."
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,0.125000,1,1,59,47,341,...,9,30,4.796610,0,53,53,0,0,1,"{'provided': 1, 'home': 1, 'service,': 1, 'dur..."


### Parts of Speech Tagging (POS Tag)
#### Generally , while working with reviewd and Tweets , we can ignore/remove the Nouns(All kinds of them) which will help in reducing the Terms.

In [147]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

data['noun_count'] = data['Reviews'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['Reviews'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count']  = data['Reviews'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count']  = data['Reviews'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['Reviews'].apply(lambda x: check_pos_tag(x, 'pron')) 

import nltk
nltk.download('all')

import nltk
nltk.download('averaged_perceptron_tagger')

In [148]:
data.head()

,Date,Stars,Reviews,BankName,sentiment,sentiment_type,count_sent,count_word,count_unique_word,count_letters,...,alphabetetics,Spaces,words_ends_with_et,words_start_with_no,wordcounts,noun_count,verb_count,adj_count,adv_count,pron_count
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,0.533333,1,1,19,19,126,...,15,0,0,0,"{'Thank': 1, 'and': 1, 'you!': 1, 'helped': 1,...",8,3,1,1,3
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,0.453333,1,1,25,23,159,...,21,0,0,0,"{'Thank': 1, 'much': 1, 'service,': 1, 'produc...",7,3,4,2,4
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,-0.033231,0,1,79,64,462,...,73,0,0,1,"{'money.': 1, 'Brad': 1, 'past': 1, '&': 1, 'a...",24,10,5,8,8
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,0.093740,0,1,108,78,605,...,101,0,0,1,"{'money.': 1, 'was': 1, 'dealing': 2, 'extra':...",30,17,8,7,11
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,0.125000,1,1,59,47,341,...,53,0,0,1,"{'provided': 1, 'home': 1, 'service,': 1, 'dur...",16,9,4,4,8


In [149]:
data.adj_count.value_counts()

3     69
2     64
4     51
5     47
6     41
7     40
1     36
9     30
8     25
13    16
10    15
12     9
11     9
0      7
17     7
21     6
14     5
15     4
16     4
18     4
23     3
19     2
20     2
24     2
27     2
35     2
34     1
22     1
32     1
Name: adj_count, dtype: int64

### After Data Cleaning ,Train Test Split

In [150]:
# define X and y
X = data.Reviews
y = data.sentiment_type

# split the new DataFrame into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(378,)
(127,)
(378,)
(127,)


In [151]:
data.shape

(505, 29)

In [152]:
#Abbrevations and Words correction
def clean_text(text):
    text = text.lower()
    text = text.strip()
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"[-()\"#/@;:{}`+=~|.!?,'0-9]", "", text)
    return(text)

In [153]:
stop = set(nltk.corpus.stopwords.words('english'))

### Data CLeaning

In [154]:
import string
def pre_process(text):
    text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #Removing stop words
    return(text)

# Without creating sentiment_type variable 
#NB  for L1, Count Vectors:  0.889763779527559
#NB  for L1, WordLevel TF-IDF:  0.952755905511811
#NB  for L1, N-Gram Vectors:  0.8188976377952756
#NB for L1, CharLevel Vectors:  0.8031496062992126
    
#LR  for L1, Count Vectors:  0.8661417322834646
#LR  for L1, WordLevel TF-IDF:  0.9448818897637795
#LR  for L1, N-Gram Vectors:  0.8582677165354331
#LR for L1, CharLevel Vectors:  0.8031496062992126
    
#SVC  for L1, Count Vectors:  0.952755905511811
#SVC  for L1, WordLevel TF-IDF:  0.968503937007874
#SVC  for L1, N-Gram Vectors:  0.9606299212598425
#SVC for L1, CharLevel Vectors:  0.889763779527559

In [155]:
X_train = X_train.apply(lambda x: clean_text(x))
X_test = X_test.apply(lambda x: clean_text(x))

In [156]:
X_train=pre_process(X_train)
X_test=pre_process(X_test)

import nltk
nltk.download('wordnet')

### Defining CountVectorizer with Unigrams

In [157]:
#Train
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1, 1 ), 
                             min_df=5, #minimum document frequency
                             encoding='latin-1' ,
                             max_features=800)
xtrain_count = count_vect.fit_transform(X_train) # applying fit_transform on X_train

In [158]:
dtm=xtrain_count.toarray()

In [159]:
dtm

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [160]:
count_vect.get_feature_names()

['aaron',
 'able',
 'absolutely',
 'accept',
 'accommodating',
 'account',
 'accurate',
 'across',
 'actual',
 'adam',
 'additional',
 'advice',
 'agent',
 'agreed',
 'alex',
 'almost',
 'along',
 'already',
 'also',
 'always',
 'amazing',
 'american',
 'amount',
 'another',
 'answer',
 'answered',
 'answering',
 'answers',
 'anyone',
 'anything',
 'application',
 'apply',
 'appraisal',
 'appraiser',
 'appreciate',
 'appreciated',
 'approved',
 'around',
 'ask',
 'asked',
 'asking',
 'aspects',
 'attention',
 'available',
 'away',
 'awesome',
 'back',
 'bad',
 'balance',
 'bank',
 'banks',
 'barrett',
 'based',
 'became',
 'beginning',
 'beneficial',
 'best',
 'better',
 'beyond',
 'big',
 'bob',
 'brent',
 'broker',
 'business',
 'buy',
 'buyer',
 'buyers',
 'buying',
 'ca',
 'call',
 'called',
 'calling',
 'calls',
 'calm',
 'came',
 'cannot',
 'cant',
 'capital',
 'care',
 'causing',
 'certainly',
 'change',
 'changed',
 'check',
 'chose',
 'chris',
 'circumstances',
 'clear',
 'cli

In [161]:
dtm1=pd.DataFrame(dtm)

In [162]:
dtm1.columns=count_vect.get_feature_names()

In [163]:
dtm1.head()  # Document Term Matrix or Bag OF Words

,aaron,able,absolutely,accept,accommodating,account,accurate,across,actual,adam,...,working,works,would,wouldnt,writing,wrong,wyndham,year,years,yet
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Applying TF-IDF on CountVectorizer transformed Data

In [164]:
# Applying tf-idf transformer on train data 
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(xtrain_count) # xtrain_count is Count_Vectorizer applied on X_train

#Test
#count_vect = CountVectorizer()
xtest_count = count_vect.transform(X_test)

#tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.transform(xtest_count)

In [165]:
dtm2=pd.DataFrame(X_train_tfidf.toarray(), columns=count_vect.get_feature_names())

In [166]:
dtm2.head(10)

,aaron,able,absolutely,accept,accommodating,account,accurate,across,actual,adam,...,working,works,would,wouldnt,writing,wrong,wyndham,year,years,yet
0,0.0,0.0,0.000000,0.133259,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.123737,0.0,0.0,0.0,0.0,0.096250,0.0
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.194406,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.000000,0.130702,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.121363,0.0,0.0,0.0,0.0,0.094403,0.0
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.084774,0.175643,0.0,0.0,0.0,0.0,0.000000,0.0
6,0.0,0.0,0.000000,0.000000,0.0,0.0,0.256938,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
7,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.068074,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
8,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
9,0.0,0.0,0.228774,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


### Creating BiGRams and Character Level TF-IDF as selection parameters for BEst Accuracy in Model Building

In [167]:
# ngram level tf-idf [ Fit and transform working is different here]

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 2), max_features=800)
tfidf_vect_ngram.fit(data['Reviews'])   # Applying fit on target variable i.e data["Reviews"]
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)   # then , transform on X_train and X_test
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [168]:
# characters level tf-idf [Same as above] ,Here analyzer = "char"

tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=800)
tfidf_vect_ngram_chars.fit(data['Reviews'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test)

### Topic MOdelling

In [169]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(X_train_tfidf)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

In [170]:
# view the topic models
n_top_words = 25
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

topic_summaries

['process loan great us would home time work recommend team questions read closing mortgage rate experience best service always made easy get everything first nasb',
 'helpful able quickly concerns industry knowledgeable fantastic experience pleasure rates answer throughout closing fast mortgage market poor never fred provided hold made questions calm team',
 'rate kory decision financial loan make took stayed calls felt via werent stars date joey time option different communicate concerns could gave quick comfortable almost',
 'adam refi everyone fast recommend helpful would possible service online smooth best documents loan process hesitate work thru confident detail attention talking transaction jason june',
 'quick agent thorough pleased simple completed little things received well close working service actual twice documentation accurate paid monthly necessary amount attention information times provided',
 'lender unprofessional number list previous contacted phone ask alex better

In [171]:
frequency_words_wo_stop= {}
for data in data['Reviews']:
    tokens = nltk.wordpunct_tokenize(data.lower())
    for token in tokens:
        if token.lower() not in stop:
            if token in frequency_words_wo_stop:
                count = frequency_words_wo_stop[token]
                count = count + 1
                frequency_words_wo_stop[token] = count
            else:
                frequency_words_wo_stop[token] = 1

In [172]:
frequency_words_wo_stop

{'great': 139,
 'job': 25,
 ',': 1135,
 'wyndham': 16,
 'capital': 12,
 '!': 222,
 'person': 17,
 'professional': 70,
 'helped': 35,
 'us': 207,
 'move': 12,
 'refinance': 70,
 'process': 253,
 'smoothly': 15,
 '.': 1976,
 'thank': 50,
 'matthew': 3,
 'richardson': 1,
 'helpful': 56,
 'find': 19,
 'correct': 1,
 'product': 10,
 'mortgage': 147,
 'much': 38,
 'excellent': 30,
 'service': 99,
 'past': 13,
 'experience': 110,
 'would': 246,
 'without': 29,
 'question': 7,
 'use': 48,
 'needed': 29,
 '!!': 24,
 'went': 70,
 'beyond': 25,
 'extra': 19,
 'mile': 8,
 'right': 28,
 'wrong': 7,
 'encountered': 7,
 'servicer': 3,
 'dealing': 14,
 'previous': 13,
 'loan': 313,
 'pulled': 4,
 'together': 8,
 'found': 16,
 'viable': 3,
 'option': 8,
 'ultimately': 5,
 'saved': 7,
 'money': 22,
 'highly': 65,
 'recommend': 156,
 'brad': 7,
 'thomka': 3,
 'team': 113,
 'needs': 27,
 'sincerest': 2,
 'thanks': 61,
 'ed': 2,
 '&': 30,
 'lind': 1,
 'beginning': 32,
 'started': 17,
 'stressful': 24,
 'ti

### WORDCLOUD (MOST REPETITIVE WORDS) 

In [173]:
from wordcloud import WordCloud ,STOPWORDS

In [174]:
wordcloud = WordCloud(stopwords=[]).generate(' '.join(X_train.tolist()))

%matplotlib inline
fig = plt.figure(figsize=(200,100))
plt.imshow(wordcloud)

### Function defined for Model Building and finding the Predictions for the Test Data

In [175]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,  valid_y, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)
    #return(metrics.classification_report(predictions,valid_y))

### Naive Bayes (Gave HIghest Accuracy for Word-Level TF-IDF)

In [176]:
#Naive Bayes
# Naive Bayes on Count Vectors and TF-IDF
accuracy_L1 = train_model(naive_bayes.MultinomialNB(), X_train_tfidf, y_train, X_test_tfidf, y_test)

print("NB  for L1, Count Vectors: ", accuracy_L1)



# Naive Bayes on Word Level TF IDF Vectors
accuracy_L1 = train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xtest_count, y_test)
print("NB  for L1, WordLevel TF-IDF: ", accuracy_L1)



# Naive Bayes on Ngram Level TF IDF Vectors
accuracy_L1 = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, y_test)
print("NB  for L1, N-Gram Vectors: ", accuracy_L1)



# Naive Bayes on Character Level TF IDF Vectors
accuracy_L1 = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars, y_test)
print("NB for L1, CharLevel Vectors: ", accuracy_L1)

NB  for L1, Count Vectors:  0.8267716535433071
NB  for L1, WordLevel TF-IDF:  0.8740157480314961
NB  for L1, N-Gram Vectors:  0.8110236220472441
NB for L1, CharLevel Vectors:  0.8110236220472441


### Logistic Regression( Gave HIghest Accuracy with Word-Level TF-IDF)    

In [177]:
#Logistic Regression
# Logistic Regression on Count Vectors and TF-IDF
accuracy_L1 = train_model(LogisticRegression(), X_train_tfidf, y_train, X_test_tfidf, y_test)
print("LR  for L1, Count Vectors: ", accuracy_L1)



# Logistic Regression on Word Level TF IDF Vectors
accuracy_L1 = train_model(LogisticRegression(), xtrain_count, y_train, xtest_count, y_test)
print("LR  for L1, WordLevel TF-IDF: ", accuracy_L1)



# Logistic Regression on Ngram Level TF IDF Vectors
accuracy_L1 = train_model(LogisticRegression(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, y_test)
print("LR  for L1, N-Gram Vectors: ", accuracy_L1)



# Logistic Regression on Character Level TF IDF Vectors
accuracy_L1 = train_model(LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars, y_test)
print("LR for L1, CharLevel Vectors: ", accuracy_L1)

LR  for L1, Count Vectors:  0.8110236220472441
LR  for L1, WordLevel TF-IDF:  0.889763779527559
LR  for L1, N-Gram Vectors:  0.8110236220472441
LR for L1, CharLevel Vectors:  0.8110236220472441


### LInear SVC (GAve highest Accuracy with Count Vectorizers)

In [178]:
#Linear SVC
# Linear SVC on Count Vectors and TF-IDF
accuracy_L1 = train_model(svm.LinearSVC(), X_train_tfidf, y_train, X_test_tfidf, y_test)
print("SVC  for L1, Count Vectors: ", accuracy_L1)



# Linear SVC on Word Level TF IDF Vectors
accuracy_L1 = train_model(svm.LinearSVC(), xtrain_count, y_train, xtest_count, y_test)
print("SVC  for L1, WordLevel TF-IDF: ", accuracy_L1)



# Linear SVC on Ngram Level TF IDF Vectors
accuracy_L1 = train_model(svm.LinearSVC(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, y_test)
print("SVC  for L1, N-Gram Vectors: ", accuracy_L1)



# Linear SVC on Character Level TF IDF Vectors
accuracy_L1 = train_model(svm.LinearSVC(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars, y_test)
print("SVC for L1, CharLevel Vectors: ", accuracy_L1)

SVC  for L1, Count Vectors:  0.889763779527559
SVC  for L1, WordLevel TF-IDF:  0.8346456692913385
SVC  for L1, N-Gram Vectors:  0.8818897637795275
SVC for L1, CharLevel Vectors:  0.8188976377952756
